In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from typing import Type
from torch import nn
from torch.optim import optimizer
import rasterio
import zipfile
from matplotlib import pyplot as plt
import datetime
from torchvision import transforms as transforms
import shutil
import torchmetrics
import os
from sklearn.metrics import r2_score

# --- GPU selection --- #
gpus = 7 # slot number (e.g., 3), no gpu use -> write just ' '
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=str(gpus)
#os.environ["TORCH_CUDA_ARCH_LIST"]="3.5"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
def minmax(array : Type[np.ndarray], dim = 0):
    min = np.min(array, axis=dim)
    max = np.max(array, axis=dim)
    array = (array-min)/(max-min)
    return array

In [ ]:
lidar_image = rasterio.open('../Data/N12/N12_lidar.tif').read()
lidar_array = np.array(lidar_image)
lidar_array = minmax(lidar_array, dim=(0,1))

lidar_1n_image = rasterio.open('../Data/N12/N12_lidar_1n.tif').read()
lidar_1n_array = np.array(lidar_1n_image)
lidar_1n_array = minmax(lidar_1n_array, dim=(0,1))

lidar_nt_image = rasterio.open('../Data/N12/N12_lidar_nt.tif').read()
lidar_nt_array = np.array(lidar_nt_image)
lidar_nt_array = minmax(lidar_nt_array, dim=(0,1))

RGB2020_image = rasterio.open('../Data/N12/N12_RGB2020.tif').read()
RGB2020_array = np.array(RGB2020_image)
train_array = np.stack([lidar_array, lidar_1n_array, lidar_nt_array]).squeeze()
train_array = np.concatenate((train_array,RGB2020_array))
target_image = rasterio.open('../Data/N12/N12_newlc.tif').read()
target_array = np.array(target_image, dtype=int).squeeze()

target_array = np.where(target_array == 1, 0, target_array)
target_array = np.where(target_array == 2, 1, target_array)
target_array = np.where(target_array == 7, 2, target_array)
target_array = np.where(target_array == 8, 3, target_array)
target_array = np.where(target_array == 9, 4, target_array)
target_array = np.where(target_array == 10, 5, target_array)
target_array = np.where(target_array == 11, 6, target_array)

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, data_array : Type[np.ndarray], target_array : Type[np.ndarray], patch_size : int):
        self.data = np.zeros(((data_array.shape[1]//patch_size) * (data_array.shape[2]//patch_size), data_array.shape[0], patch_size, patch_size))
        for i in range(0,data_array.shape[1]//patch_size):
            for j in range(0,data_array.shape[2]//patch_size):
                self.data[data_array.shape[1]//patch_size*i+j,:,:,:] = data_array[:,i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size] #고치기
                
                #self.data =np.append(self.data, np.zeros(((1, data_array.shape[0], patch_size, patch_size))), axis = 0)
                #self.data = np.concatenate(self.data, np.zeros(((1, data_array.shape[0], patch_size, patch_size))))

        self.label = np.zeros(((data_array.shape[1]//patch_size) * (data_array.shape[2]//patch_size),data_array.shape[0]+1))
        
        for k in range(0,data_array.shape[1]//patch_size):
            for l in range(0,data_array.shape[2]//patch_size):
                self.label[data_array.shape[1]//patch_size*k+l,:] = np.bincount(target_array[k*patch_size:(k+1)*patch_size, l*patch_size:(l+1)*patch_size].reshape(-1), minlength=7)/(patch_size*patch_size)
        self.data = torch.as_tensor(self.data).float()
        self.label = torch.as_tensor(self.label).float()

        self.data[:,3:6,:,:] = self.data[:,3:6,:,:]/255

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        sample = torch.as_tensor(self.data[idx,:,:,:]).float()
        label = torch.as_tensor(self.label[idx,:]).float()
        return sample, label


In [ ]:
class TrainDataset2(Dataset):
    def __init__(self, data_array : Type[np.ndarray], target_array : Type[np.ndarray], patch_size : int, is_evaluating : bool = False, is_validating : bool = False, rotate : bool = False, train_ratio : float = 0.8):
        self.is_validating = is_validating
        self.is_evaluating = is_evaluating
        seed = 386579

        self.data = np.zeros(((data_array.shape[1]//patch_size) * (data_array.shape[2]//patch_size), data_array.shape[0], patch_size, patch_size))

        for i in range(0,data_array.shape[1]//patch_size):
            for j in range(0,data_array.shape[2]//patch_size):
                self.data[data_array.shape[1]//patch_size*i+j,:,:,:] = data_array[:,i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size]

        self.label = np.zeros(((data_array.shape[1]//patch_size) * (data_array.shape[2]//patch_size),data_array.shape[0]+1))

        for k in range(0,data_array.shape[1]//patch_size):
            for l in range(0,data_array.shape[2]//patch_size):
                self.label[data_array.shape[1]//patch_size*k+l,:] = np.bincount(target_array[k*patch_size:(k+1)*patch_size, l*patch_size:(l+1)*patch_size].reshape(-1), minlength=7)/(patch_size*patch_size)

        if not is_evaluating:
            if rotate:
                for i in range(2):
                    rotated_data = np.rot90(self.data, k=i+1, axes=(-2, -1))
                    self.data = np.concatenate((self.data, rotated_data), axis=0)
                    rotated_data = self.label
                    self.label = np.concatenate((self.label, rotated_data), axis=0)
                    
                        
                #self.data =np.append(self.data, np.zeros(((1, data_array.shape[0], patch_size, patch_size))), axis = 0)
                #self.data = np.concatenate(self.data, np.zeros(((1, data_array.shape[0], patch_size, patch_size))))

        train_size = int(self.data.shape[0]*train_ratio)
        index_array = np.random.RandomState(seed=seed).permutation(self.data.shape[0])
        self.train_index = index_array[0:train_size]
        self.valid_index = index_array[train_size:index_array.shape[0]]
        
        self.data = torch.as_tensor(self.data).float()
        self.label = torch.as_tensor(self.label).float()

        self.data[:,3:6,:,:] = self.data[:,3:6,:,:]/255

    def __len__(self):
        if self.is_evaluating:
            return self.data.shape[0]

        if self.is_validating:
            return self.valid_index.shape[0]
        else:
            return self.train_index.shape[0]

    def __getitem__(self, idx):
        if self.is_evaluating:
            sample = torch.as_tensor(self.data[idx,:,:,:]).float()
            label = torch.as_tensor(self.label[idx,:]).float()
            return sample, label
        
        if self.is_validating:
            sample = torch.as_tensor(self.data[self.valid_index[idx],:,:,:]).float()
            label = torch.as_tensor(self.label[self.valid_index[idx],:]).float()
        else:
            sample = torch.as_tensor(self.data[self.train_index[idx],:,:,:]).float()
            label = torch.as_tensor(self.label[self.train_index[idx],:]).float()

        return sample, label
